# Query playground

In [1]:
import os
os.getcwd()

'/Users/will/Documents/github/cartar/alpine-fng.github.io/data'

In [4]:
import pandas as pd
import numpy as np
import sqlite3

from queries import get_race_data, get_point_total, get_races_list, audit_df, get_race_years

In [5]:
# Set the database path to a location with write permissions
db_path = '../race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [4]:
race_list = get_races_list(conn)

In [11]:
df = pd.read_sql("""
    SELECT strftime('%Y', race_date) as year
    FROM Races
    GROUP BY 1
    ORDER BY CAST(year AS INT) DESC
    """,
    conn
)

In [12]:
df

,year
0,2024
1,2023
2,2022
3,2020
4,2019
5,2018
6,2017
7,2016
8,2015
9,2014


In [35]:
years =['2020', '2022', '2023', '2024']
races_by_year = {}

In [36]:
#race_list = get_races_list(CONN)

races_by_year = {}
for y in years:
    races_for_year = race_list[race_list['year'] == y]
    races_list = races_for_year.to_dict('records')
    races_by_year[y] = races_list

In [37]:
races_metadata = {
    "years": years,
    "races": races_by_year
}

In [38]:
races_metadata

{'years': ['2020', '2022', '2023', '2024'],
 'races': {'2020': [{'race_id': 2032,
    'race_date': '2020-02-29 00:00:00',
    'description': '2020 F&G Race#4',
    'year': '2020'},
   {'race_id': 2031,
    'race_date': '2020-02-16 00:00:00',
    'description': '2020 F&G Race#3',
    'year': '2020'},
   {'race_id': 2030,
    'race_date': '2020-02-02 00:00:00',
    'description': '2020 F&G Race#2',
    'year': '2020'},
   {'race_id': 2029,
    'race_date': '2020-01-19 00:00:00',
    'description': '2020 F&G Race#1',
    'year': '2020'}],
  '2022': [{'race_id': 3035,
    'race_date': '2022-12-31 00:00:00',
    'description': '2023 Time Trials',
    'year': '2022'},
   {'race_id': 2037,
    'race_date': '2022-02-27 00:00:00',
    'description': '2022F&G#4',
    'year': '2022'},
   {'race_id': 2036,
    'race_date': '2022-02-21 00:00:00',
    'description': '2022F&G#3',
    'year': '2022'},
   {'race_id': 2035,
    'race_date': '2022-01-22 00:00:00',
    'description': '2022F&G#2',
    'yea

In [46]:
race_results = []
for year in races_metadata["years"]:
        for race_meta in races_metadata["races"][year]:
            race_df = get_race_data(
                'best_time',
                race_meta["race_id"],
                year,
                conn
            )
            race_df["race_id"] = race_meta["race_id"]
            race_results.append(race_df)

In [48]:
race_results[1]

,bib,name,discipline,team,tier,run1,run2,best_time,points,race_id
0,None,None,SKI,None,None,30.77,None,30.77,None,2031
1,None,None,SKI,None,None,30.93,None,30.93,None,2031
2,None,None,SKI,None,None,31.16,None,31.16,None,2031
3,None,None,SKI,None,None,31.63,None,31.63,None,2031
4,None,None,SKI,None,None,33.03,None,33.03,None,2031
...,...,...,...,...,...,...,...,...,...,...
83,None,None,SKI,None,None,51.72,None,51.72,None,2031
84,None,None,SKI,None,None,52.77,None,52.77,None,2031
85,None,None,SKI,None,None,55.86,None,55.86,None,2031
86,None,None,SKI,None,None,62.04,None,62.04,None,2031


In [53]:
if race_df.shape[0] > 0:
    print("yes")

yes


In [50]:
foo = pd.concat(race_results)
foo.shape

/var/folders/cp/tk2rjzwj4n76t_7mcr1f2y640000gn/T/ipykernel_31787/4233780671.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  foo = pd.concat(race_results)


(1346, 10)

In [51]:
foo

,bib,name,discipline,team,tier,run1,run2,best_time,points,race_id
0,NaN,None,SKI,None,NaN,43.00,NaN,43.00,NaN,2032
1,NaN,None,SKI,None,NaN,43.54,NaN,43.54,NaN,2032
2,NaN,None,SKI,None,NaN,43.94,NaN,43.94,NaN,2032
3,NaN,None,SKI,None,NaN,44.62,NaN,44.62,NaN,2032
4,NaN,None,SKI,None,NaN,44.91,NaN,44.91,NaN,2032
...,...,...,...,...,...,...,...,...,...,...
99,60.0,Robert Hawtin,SKI,Mike McTaggart,8.0,9999.00,9999.0,9999.00,0.0,240114
100,39.0,Stephen Lewis,SKI,Joanna Perreault,5.0,9999.00,9999.0,9999.00,0.0,240114
101,109.0,Mackenzie Hamilton,SNBD,Joanna Perreault,12.0,9999.00,9999.0,9999.00,0.0,240114
102,50.0,Adam Grossman,SKI,Adam Grossman,7.0,9999.00,9999.0,9999.00,0.0,240114


# HERE! Need to clean up this race pull!! Then save in json, and fix up the webpage ...
# including the creation of two tables, one for each discipline

In [22]:
race_data = get_race_data('best_time', '240218', 2024, conn)
race_data[race_data.discipline == 'SNBD']

,bib,name,discipline,team,tier,run1,run2,best_time,points
4,120.0,Hannah Kilmer Choi,SNBD,Will Carter,13.0,25.41,25.34,25.34,8.0
5,101.0,Joel Farber,SNBD,Will Carter,11.0,9998.00,26.47,26.47,8.0
6,114.0,Sacha Fiand,SNBD,Mitch Perreault,12.0,26.52,27.89,26.52,8.0
7,109.0,Mackenzie Hamilton,SNBD,Joanna Perreault,12.0,26.61,29.83,26.61,7.0
8,119.0,Jenna Livingston,SNBD,Stephanie Coward,13.0,27.78,27.40,27.40,7.0
9,121.0,Graham Ramshaw,SNBD,Graham Ramshaw,13.0,28.53,28.58,28.53,6.0
10,107.0,Terence Woodside,SNBD,Mike McTaggart,11.0,28.72,9999.00,28.72,7.0
11,123.0,Bernard Oegema,SNBD,Don French,13.0,28.97,30.17,28.97,5.0
12,116.0,Melanie Leistner Lavoie,SNBD,Mike McTaggart,13.0,29.41,29.00,29.00,4.0
13,102.0,Sarah Ledwidge,SNBD,Joanna Perreault,11.0,30.58,33.57,30.58,6.0


In [23]:
race_data[race_data.discipline == 'SKI']

,bib,name,discipline,team,tier,run1,run2,best_time,points
22,75.0,Nicholas Balan,SKI,Don French,10.0,37.17,37.64,37.17,8.0
23,80.0,Will Stewart,SKI,Adam Grossman,10.0,41.86,37.43,37.43,7.0
24,77.0,Brandon Hune,SKI,Mitch Perreault,10.0,37.55,37.61,37.55,6.0
25,69.0,William Cartar,SKI,Will Carter,9.0,37.65,9998.00,37.65,8.0
26,79.0,Thomas Daly,SKI,Joanna Perreault,10.0,37.67,38.14,37.67,5.0
...,...,...,...,...,...,...,...,...,...
97,40.0,Earl Duffy,SKI,Adam Grossman,5.0,9999.00,9999.00,9999.00,0.0
98,46.0,Craig Pitman,SKI,Adam Grossman,6.0,9999.00,9999.00,9999.00,0.0
99,50.0,Adam Grossman,SKI,Adam Grossman,7.0,9999.00,9999.00,9999.00,0.0
100,66.0,Jeff Cox,SKI,Adam Grossman,9.0,9999.00,9999.00,9999.00,0.0
